In [14]:
import numpy as np
import pandas as pd
import scipy
from scipy.stats import norm

In [41]:
data = [['Chlorpromazine',75,26],
       ['Dimenhydrinate',85,52],
       ['Pentobarbital 100mg',67,35],
       ['Pentobarbital 150mg',85,37]]

In [42]:
df = pd.DataFrame(data)
df.columns = ['Drug','Num_Patients','Inc_Nausea']

In [43]:
df

,Drug,Num_Patients,Inc_Nausea
0,Chlorpromazine,75,26
1,Dimenhydrinate,85,52
2,Pentobarbital 100mg,67,35
3,Pentobarbital 150mg,85,37


In [44]:
def z_value(pats,naus):
    p_1 = 45*1.0 / 80
    p_2 = naus*1.0 / pats
    std = np.sqrt( p_1*(1-p_1) / 80 + p_2*(1-p_2) / pats )
    return (np.abs(p_2-p_1)/std)
    

In [45]:
z_value(75,26)

2.7643637780027186

In [46]:
p_val = lambda z: 2*(1-norm.cdf(z))

In [47]:
df['p_value'] = df.apply(lambda x: p_val(z_value(x['Num_Patients'],x['Inc_Nausea'])) ,1)

In [48]:
df

,Drug,Num_Patients,Inc_Nausea,p_value
0,Chlorpromazine,75,26,0.005703
1,Dimenhydrinate,85,52,0.520232
2,Pentobarbital 100mg,67,35,0.626664
3,Pentobarbital 150mg,85,37,0.099639


# Bonferroni

In [49]:
df['bonferroni_rej'] = df.apply(lambda x: x['p_value']/4 < 0.05/4 ,1)

In [50]:
df

,Drug,Num_Patients,Inc_Nausea,p_value,bonferroni_rej
0,Chlorpromazine,75,26,0.005703,True
1,Dimenhydrinate,85,52,0.520232,False
2,Pentobarbital 100mg,67,35,0.626664,False
3,Pentobarbital 150mg,85,37,0.099639,False


# Benjamini Hochberg

In [51]:
df = df.sort_values('p_value')

In [52]:
df

,Drug,Num_Patients,Inc_Nausea,p_value,bonferroni_rej
0,Chlorpromazine,75,26,0.005703,True
3,Pentobarbital 150mg,85,37,0.099639,False
1,Dimenhydrinate,85,52,0.520232,False
2,Pentobarbital 100mg,67,35,0.626664,False


In [53]:
# assume the p values are independant.
m = len(df)

In [54]:
df.iloc[1]['p_value']

0.099639233418202355

In [55]:
rej_vec = []
for i in range(m):
    l_i = (i+1)*0.05/m
    if df.iloc[i]['p_value'] < l_i:
        rej_vec.append(True)
    else:
        rej_vec.append(False)

In [56]:
rej_vec

[True, False, False, False]

In [57]:
print( "t = " + str(np.argmin(rej_vec)))

t = 1


# Reject all null hypothesis for which $P_{i} \leq t$